In [2]:
!pip install keras

     |████████████████████████████████| 1.3 MB 3.0 MB/s eta 0:00:01


In [4]:
!pip install tensorflow

     |████████████████████████████████| 489.6 MB 22 kB/s s eta 0:00:01
     |████████████████████████████████| 463 kB 93.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 44 kB/s s eta 0:00:01
     |████████████████████████████████| 57 kB 993 kB/s  eta 0:00:01
     |████████████████████████████████| 3.9 MB 110.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 73.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 95.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 28.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 35.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 554 kB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 96.5 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 1.4 MB/s  eta 0:00:01
     |████████████████████████████

In [1]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils.np_utils import to_categorical
nltk.download('punkt')
# from doc3 import training_doc3
dataset = open("mahabharata.csv", "r").read()
dataset=dataset[:100000]
cleaned = re.sub(r'\W+', ' ', dataset).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


[nltk_data] Downloading package punkt to /Users/shubham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])

(16986, 3)

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

In [12]:

#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))

model.add(LSTM(60,return_sequences=True))
model.add(LSTM(60))
model.add(Dropout(0.1))
model.add(Dense(60,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 3, 3)              9441      
                                                                 
 lstm_6 (LSTM)               (None, 3, 60)             15360     
                                                                 
 lstm_7 (LSTM)               (None, 60)                29040     
                                                                 
 dropout_1 (Dropout)         (None, 60)                0         
                                                                 
 dense_5 (Dense)             (None, 60)                3660      
                                                                 
 dense_6 (Dense)             (None, 3147)              191967    
                                                                 
Total params: 249,468
Trainable params: 249,468
Non-tr

In [4]:
model=load_model('mymodel.h5')

In [16]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    pred_word = tokenizer.index_word[i]
    print("Next word suggestion:",pred_word)

The sons of Div were
Next word suggestion: brihadbhanu
Next word suggestion: kettle
Next word suggestion: come


In [14]:
dataset[10000:20000]

' Babu Aghore Nath Banerjee also has from time to time, rendered me valuable assistance in clearing my difficulties."\n"Gigantic as the work is, it would have been exceedingly difficult for me to go on with it if I had not been encouraged by Sir Stuart Bayley, Sir Auckland Colvin, Sir Alfred Croft, and among Oriental scholars, by the late lamented Dr. Reinhold Rost, and Mons. A. Barth of Paris. All these eminent men know from the beginning that the translation was proceeding from my pen. Notwithstanding the enthusiasm, with which my poor friend, Pratapa Chandra Roy, always endeavoured to fill me. I am sure my energies would have flagged and patience exhausted but for the encouraging words which I always received from these patrons and friends of the enterprise."\n"Lastly, I should name my literary chief and friend, Dr. Sambhu C. Mookherjee. The kind interest he took in my labours, the repeated exhortations he addressed to me inculcating patience, the care with which he read every fasci